<h1> <center> CS329e Darwn Project </center> </h1> 

<h2> <center> Name of Project </center> </h2>

<center> Explanation of what the project is and it's importance <center>

Links to use markdown and command mode to make this more readible (https://gist.github.com/kidpixo/f4318f8c8143adee5b40) (https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

Example: Press esc (enables command mode) then press b to make a cell below, press esc then m to allow it to be text and not code. Use shift+enter in order to move to turn it into nice text and move to next cell

## Import libraries

In [53]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk

## Setup

In [54]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('thomasmendez@utexas.edu', 'ZgQ4ktGjF2')

if not status:
    print(msg)

### **Data Paths**

**Smart Locations Database**

In [55]:
path = './datasets'
dataset_name = 'SmartLocationDB.csv'
smartLocation = pd.read_csv(os.path.join(path,dataset_name))
smartLocation.head()
#smartLocations

,OBJECTID,GEOID10,TRFIPS,CFIPS,SFIPS,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_EMP,...,D5cr,D5cri,D5ce,D5cei,D5dr,D5dri,D5de,D5dei,Shape_Length,Shape_Area
0,1,10059505002,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.66,4.24,0.68,-99999.0,-99999.0,-99999.0,-99999.0,0.846952,0.018273
1,2,10059505001,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.67,4.70,0.75,-99999.0,-99999.0,-99999.0,-99999.0,0.521148,0.005095
2,3,10059502001,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.30,2.60,0.42,-99999.0,-99999.0,-99999.0,-99999.0,0.660835,0.018740
3,4,10059502002,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.01,0.27,2.58,0.41,-99999.0,-99999.0,-99999.0,-99999.0,0.695057,0.020340
4,5,10059504002,950400,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.32,3.35,0.54,-99999.0,-99999.0,-99999.0,-99999.0,0.499117,0.008829


**Cencus Tract Data**

In [56]:
path = './datasets'
dataset_name = 'SocialExplorer.csv'
socialExplorer = pd.read_csv(os.path.join(path,dataset_name), encoding='cp1252')
socialExplorer.head()
#cencus_DB

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,SE_T128_001,SE_T128_002,SE_T128_009,SE_T128_010,SE_T128_003,SE_T128_004,SE_T128_005,SE_T128_006,SE_T128_007,SE_T128_008
0,10010201001,15000US010010201001,"Block Group 1, Census Tract 201, Autauga Count...","Block Group 1, Census Tract 201, Autauga Count...",al,150,0,ACSSF,5837,NaN,...,177,162,157,5,0,9,0,0,0,6
1,10010201002,15000US010010201002,"Block Group 2, Census Tract 201, Autauga Count...","Block Group 2, Census Tract 201, Autauga Count...",al,150,0,ACSSF,5838,NaN,...,652,598,583,15,22,5,0,0,27,0
2,10010202001,15000US010010202001,"Block Group 1, Census Tract 202, Autauga Count...","Block Group 1, Census Tract 202, Autauga Count...",al,150,0,ACSSF,5839,NaN,...,475,475,406,69,0,0,0,0,0,0
3,10010202002,15000US010010202002,"Block Group 2, Census Tract 202, Autauga Count...","Block Group 2, Census Tract 202, Autauga Count...",al,150,0,ACSSF,5840,NaN,...,451,451,439,12,0,0,0,0,0,0
4,10010203001,15000US010010203001,"Block Group 1, Census Tract 203, Autauga Count...","Block Group 1, Census Tract 203, Autauga Count...",al,150,0,ACSSF,5841,NaN,...,1098,1052,1002,50,3,0,0,0,0,43


## Combine Data

**Combine data from the Smart Locations Database and Census Tract Data. Both datasets are for the entire U.S.**

Both datasets have a GEOID that is used to represent a geographic location

Some of the most common administrative/legal and statistical geographic entities with unique GEOIDs include states, counties, congressional districts, core based statistical areas (metropolitan and micropolitan areas), census tracts, block groups and census blocks. <br>

*Note: GEOIDs are numeric codes that uniquely identify all administrative/legal and statistical geographic areas for which the Census Bureau tabulates data.*

**Get Social Explorer Data**

Social Expolorer dataset with column name and description. Get the mode share 

| Column # | Column Name | Description                              |
|----------|-------------|------------------------------------------|
| 0        | Geo_FIPS    | Geographic Identifier                    |
| 55       | SE_T128_001 | Workers 16 Years and Over:               |
| 56       | SE_T128_002 | Car, Truck, or Van                       |
| 57       | SE_T128_009 | Drove Alone                              |
| 58       | SE_T128_010 | Carpooled                                |
| 59       | SE_T128_003 | Public Transportation (Includes Taxicab) |
| 60       | SE_T128_004 | Motorcycle                               |
| 61       | SE_T128_005 | Bicycle                                  |
| 62       | SE_T128_006 | Walked                                   |
| 63       | SE_T128_007 | Other Means                              |
| 64       | SE_T128_008 | Worked At Home                           |

In [57]:
#includes only relevant columns (modeShareSplit and Geo_FIPS code) from Social Explorer

modeShareAll = socialExplorer.iloc[:,[0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]]
#modeShareAll.head()
#print(modeShareAll.columns.values)

#creates new dataframe with % SOV
modeShareSOV = modeShareAll.iloc[:,[0,1,2]]
modeShareSOV.head()
percSOV = modeShareAll['SE_T128_009']/modeShareAll['SE_T128_001']
modeShareSOV['percSOV'] = percSOV

#new dataframe with Geo_FIPS and SOV only (ID and class)
print(list(modeShareSOV))
modeShareSOVPerc = modeShareSOV.iloc[:,[0,3]]
print(modeShareSOVPerc.head())

['Geo_FIPS', 'SE_T128_001', 'SE_T128_002', 'percSOV']
      Geo_FIPS   percSOV
0  10010201001  0.887006
1  10010201002  0.894172
2  10010202001  0.854737
3  10010202002  0.973392
4  10010203001  0.912568


C:\Anaconda3_2018_12\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


**Get Smart Location Data**

List of features from Smart Location

In [58]:
#contains features
#smartLocation.head()
list(smartLocation.columns.values)

['OBJECTID',
 'GEOID10',
 'TRFIPS',
 'CFIPS',
 'SFIPS',
 'CSA',
 'CSA_Name',
 'CBSA',
 'CBSA_Name',
 'CBSA_EMP',
 'CBSA_POP',
 'CBSA_WRK',
 'COUNTHU10',
 'TOTPOP10',
 'HH',
 'P_WRKAGE',
 'AUTOOWN0',
 'PCT_AO0',
 'AUTOOWN1',
 'PCT_AO1',
 'AUTOOWN2P',
 'PCT_AO2P',
 'WORKERS',
 'R_LOWWAGEWK',
 'R_MEDWAGEWK',
 'R_HIWAGEWK',
 'R_PCTLOWWAGE',
 'EMPTOT',
 'E5_RET10',
 'E5_OFF10',
 'E5_IND10',
 'E5_SVC10',
 'E5_ENT10',
 'E8_RET10',
 'E8_OFF10',
 'E8_IND10',
 'E8_SVC10',
 'E8_ENT10',
 'E8_ED10',
 'E8_HLTH10',
 'E8_PUB10',
 'E_FEDT10',
 'E_FEDRET10',
 'E_FEDOFF10',
 'E_FEDIND10',
 'E_FEDSVC10',
 'E_FEDENT10',
 'E_LOWWAGEW',
 'E_MEDWAGEW',
 'E_HIWAGEWK',
 'E_PCTLOWWA',
 'AC_TOT',
 'AC_WATER',
 'AC_LAND',
 'AC_UNPR',
 'D1A',
 'D1B',
 'D1C',
 'D1C5_Ret10',
 'D1C5_Off10',
 'D1C5_Ind10',
 'D1C5_Svc10',
 'D1C5_Ent10',
 'D1C8_Ret10',
 'D1C8_Off10',
 'D1C8_Ind10',
 'D1C8_Svc10',
 'D1C8_Ent10',
 'D1C8_Ed10',
 'D1C8_Hlth10',
 'D1C8_Pub10',
 'D1D',
 'D1_flag',
 'D2A_JPHH',
 'D2B_E5MIX',
 'D2B_E5MIXA',
 'D2

List of features to keep 

- CSA_Name
- CBSA_Name
- P_WKRAGE
- PCT_AO0
- PCT_AO1
- PCT_A02
- R_PCTLOWWAGE

In [60]:
#contains different sets of features to keep
setOne = ['CSA_Name','CBSA_Name', 'P_WKRAGE','PCT_AO0','PCT_AO1','PCT_A02','R_PCTLOWWAGE',]
new = smartLocation[["OBJECTID","GEOID10"]].copy()

List of features to remove

In [61]:
smartLocationOnlyDs = smartLocation.drop(columns=['OBJECTID', 'GEOID10', 'TRFIPS', 'CFIPS', 'SFIPS', 'CSA','CSA_Name','CBSA','CBSA_Name','CBSA_EMP','CBSA_POP','CBSA_WRK','COUNTHU10','TOTPOP10','HH','AUTOOWN0','AUTOOWN1','AUTOOWN2P','WORKERS','R_LOWWAGEWK','R_MEDWAGEWK','R_HIWAGEWK','EMPTOT','E_FEDT10','E_FEDRET10','E_FEDOFF10','E_FEDIND10','E_FEDSVC10','E_FEDENT10','E_LOWWAGEW','E_MEDWAGEW','E_HIWAGEWK','AC_TOT','AC_WATER','AC_LAND','AC_UNPR','Shape_Length', 'Shape_Area'])

In [62]:
#clean smartLocations manually
smartLocation.head()
#smartLocationClean = smartLocation.drop(columns=['OBJECTID','CSA_Name','CBSA_Name','AUTOOWN0','AUTOOWN1','AUTOOWN2','WORKERS','R_LOWWAGEWK','R_MEDWAGEWK','R_HIWAGEWK','EMPTOT','Shape_Length','Shape_Area'])
#smartLocation.head()

,OBJECTID,GEOID10,TRFIPS,CFIPS,SFIPS,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_EMP,...,D5cr,D5cri,D5ce,D5cei,D5dr,D5dri,D5de,D5dei,Shape_Length,Shape_Area
0,1,10059505002,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.66,4.24,0.68,-99999.0,-99999.0,-99999.0,-99999.0,0.846952,0.018273
1,2,10059505001,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.67,4.70,0.75,-99999.0,-99999.0,-99999.0,-99999.0,0.521148,0.005095
2,3,10059502001,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.30,2.60,0.42,-99999.0,-99999.0,-99999.0,-99999.0,0.660835,0.018740
3,4,10059502002,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.01,0.27,2.58,0.41,-99999.0,-99999.0,-99999.0,-99999.0,0.695057,0.020340
4,5,10059504002,950400,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.32,3.35,0.54,-99999.0,-99999.0,-99999.0,-99999.0,0.499117,0.008829


**Combine Data**

In [63]:
# combine data
mergedData = pd.merge(modeShareSOVPerc, smartLocation, left_on='Geo_FIPS', right_on='GEOID10')
print(len(mergedData))
mergedData = mergedData.dropna(subset=['percSOV'])
print(len(mergedData))

220279
218937


In [64]:
#print(list(mergedData))
#for i in mergedData:
 # isNull = list(pd.isna(mergedData[i]))
  #if(isNull.count(True)!=0):
   #     print(i, "has", isNull.count(True), "null values")
  #print(isNull.count(True))
a = mergedData['CBSA_Name']
print(mergedData.columns.get_loc('CBSA_Name'))
print(len(mergedData.columns))
mergedData['CBSA_Str'] = 'a'
print(len(mergedData.columns))
#print(mergedData.iloc[:,119])
#print(mergedData.iloc[0,119])
a = list(mergedData)
#for i in range(len(mergedData)):
 # mergedData.iloc[i, 119] = str(mergedData.iloc[i,10])
print(mergedData.iloc[:, 119])
#print(b)
#b.describe()

10
119
120
0         a
1         a
2         a
3         a
4         a
5         a
6         a
7         a
8         a
9         a
10        a
11        a
12        a
13        a
14        a
15        a
16        a
17        a
18        a
19        a
20        a
21        a
22        a
23        a
24        a
25        a
26        a
27        a
28        a
29        a
         ..
220248    a
220249    a
220251    a
220252    a
220253    a
220254    a
220255    a
220256    a
220257    a
220258    a
220259    a
220260    a
220261    a
220262    a
220263    a
220264    a
220265    a
220266    a
220267    a
220268    a
220269    a
220270    a
220271    a
220272    a
220273    a
220274    a
220275    a
220276    a
220277    a
220278    a
Name: CBSA_Str, Length: 218937, dtype: object


In [65]:
#converts mergedData to csv so that it can be cleaned
mergedData.to_csv("mergedData.csv")

In [69]:
mergedDataClean = pd.read_csv("mergedData.csv").copy()
mergedDataClean = mergedDataClean.drop(columns=["CSA_Name"])
isNull = list(pd.isna(mergedDataClean["CSA"]))
#print(isNull)
print(isNull.count(True))


81060


## Data Upload and Clean

**Read dataset and view a file snippet**

After setting up the dataset path, the next step is to upload the dataset from your local device to the server. <br> In the cell below, you need to specify the dataset_name if you want to use your own data.

**Upload dataset to Darwin**

In [75]:
# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
path='./'
dataset_name='mergedData.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [77]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(path, dataset_name))
if not status:
    print(dataset)

400: BAD REQUEST - {"message": "Dataset already exists"}



In [78]:
# Preview dataset
df = pd.read_csv(os.path.join(path, dataset_name))
df.head()
#print(list(df.isnull().sum()))
#print(max(df.isnull().sum())-len(df))
#print(list(df))
#print(df.iloc[:,0])
display(df[['Geo_FIPS','CSA_Name']])

,Geo_FIPS,CSA_Name
0,10010201001,"Montgomery-Alexander City, AL"
1,10010201002,"Montgomery-Alexander City, AL"
2,10010202001,"Montgomery-Alexander City, AL"
3,10010202002,"Montgomery-Alexander City, AL"
4,10010203001,"Montgomery-Alexander City, AL"
5,10010203002,"Montgomery-Alexander City, AL"
6,10010204001,"Montgomery-Alexander City, AL"
7,10010204002,"Montgomery-Alexander City, AL"
8,10010204003,"Montgomery-Alexander City, AL"
9,10010204004,"Montgomery-Alexander City, AL"


**Clean dataset**

In [79]:
# clean dataset
target = "percSOV"
status, job_id = ds.clean_data(dataset_name, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-08T03:25:53.393156', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['8fecba69fb764d2b9df05ae7f195feb2'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-08T03:25:53.393156', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['8fecba69fb764d2b9df05ae7f195feb2'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-08T03:25:53.393156', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['8fecba69fb764d2b9df05ae7f195feb2'], 'model_name': None, 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-08T03:25:53.393156', 'endtime': '2019-04-08T03:26:24.5

## Clean Dataset Model

We will now build a model that will learn the class labels in the target column.<br> In the default cancer dataset, the target column is "Diagnosis". <br> You will have to specify your own target name for your custom dataset. <br> You can also increase max_train_time for longer training.

In [81]:
model = target + "_model0"
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

400: BAD REQUEST - {"message": "A model with this name already exists"}



In [83]:
# Train some more
status, job_id = ds.resume_training_model(dataset_names = dataset_name,
                                          model_name = model,
                                          max_train_time = '00:05')
                                          
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-08T03:28:51.321148', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 0.5565131306648254, 'generations': 9, 'dataset_names': ['mergedData.csv'], 'artifact_names': None, 'model_name': 'percSOV_model0', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-08T03:28:51.321148', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 0.5565131306648254, 'generations': 9, 'dataset_names': ['mergedData.csv'], 'artifact_names': None, 'model_name': 'percSOV_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-08T03:28:51.321148', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 'loss': 0.5565131306648254, 'generations': 9, 'dataset_names': ['mergedData.csv'], 'artifact_names': None, 'model_name': 'percSOV_model0', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-08T03:28:51.321148', 'endtime': None, 'percent_complete': 0, 'job_type': 'UpdateModel', 

In [82]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

{'status': 'Running', 'starttime': '2019-04-08T03:28:31.020332', 'endtime': None, 'percent_complete': 0, 'job_type': 'AnalyzeModel', 'loss': 0.5565131306648254, 'generations': 9, 'dataset_names': None, 'artifact_names': ['5e1daa75054b4ed59baa4805d31843cd'], 'model_name': 'percSOV_model0', 'job_error': ''}


KeyboardInterrupt: 

In [ ]:
feature_importance[:10]

# Predictions
**Perform model prediction on the the training dataset.**